In [416]:
import pandas as pd
import numpy as np

In [417]:
# Función sigmoide y su derivada
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

In [418]:
def linear_error(y_true, y_pred):
    return np.mean(y_true - y_pred)

def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

In [419]:
def initialize_weights(input_size, output_size):
    weights = np.random.randn(input_size, output_size) * 0.01
    bias = np.zeros((1, output_size))
    return weights, bias

In [420]:
def forward_pass(X, weights, bias):
    z = np.dot(X, weights) + bias
    return sigmoid(z)

In [421]:
def backpropagation(X, y, output, weights, bias, learning_rate):
    # Calcular el error
    error = output - y
    
    # Calcular los gradientes
    d_weights = np.dot(X.T, error * sigmoid_derivative(output))
    d_bias = np.sum(error * sigmoid_derivative(output), axis=0, keepdims=True)
    
    # Actualizar pesos y sesgos
    weights -= learning_rate * d_weights
    bias -= learning_rate * d_bias

    return weights, bias

In [422]:
def train(X, y, input_size, output_size, epochs, learning_rate):
    # Inicializar pesos y sesgos
    weights, bias = initialize_weights(input_size, output_size)
    
    # Entrenar por varias épocas
    for epoch in range(epochs):
        # Propagación hacia adelante
        output = forward_pass(X, weights, bias)
        
        # Calcular pérdida
        loss = linear_error(y, output)
        
        # Retropropagación
        weights, bias = backpropagation(X, y, output, weights, bias, learning_rate)
        
        # Imprimir el progreso cada 100 épocas
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    return weights, bias

In [423]:
# Predicción
def predict(X, weights, bias):
    output = forward_pass(X, weights, bias)
    # return (output > 0.3).astype(int)
    return output

In [424]:
# #x = ([precio,metroscuadrados,pisomarmol,pisomadera,hornoagas,hornoelectrico])
# #y = ([moderna,rustica,barata])

# X = np.array([
#     [150000, 120, 1, 0, 1, 0],
#     [80000, 80, 0, 1, 0, 1],
#     [200000, 150, 1, 1, 1, 1],
#     [50000, 60, 0, 0, 0, 1],
#     [120000, 100, 1, 0, 1, 0]
# ])


# y = np.array([
#     [1, 0, 0],
#     [0, 1, 1],
#     [1, 0, 0],
#     [0, 1, 1],
#     [1, 0, 0] 
# ])


In [425]:
##https://www.kaggle.com/datasets/valakhorasani/electric-vehicle-charging-patterns

dataset = pd.read_csv("ev_charging_patterns.csv")
dataset = dataset.dropna()
dataset = dataset.drop(columns=['User ID', 'Charging Station ID', 'Charging Station Location','Charging Start Time','Charging End Time','Time of Day','Day of Week','Charger Type'])
dataset.head(1)

,Vehicle Model,Battery Capacity (kWh),Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),User Type
0,BMW i3,108.463007,60.712346,0.591363,36.389181,13.087717,29.371576,86.119962,293.602111,27.947953,2.0,Commuter


In [426]:
X = dataset.drop('User Type', axis=1)
X = pd.get_dummies(X, drop_first=False).astype(int).to_numpy()

y = dataset['User Type']
y= pd.get_dummies(y, drop_first=False).astype(int).to_numpy()
y

array([[0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [427]:
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(X)
y = StandardScaler().fit_transform(y)

In [428]:
# Parámetros
input_size = X.shape[1]
output_size = y.shape[1]
epochs = 3000
learning_rate = 0.01

In [429]:
# Entrenar el modelo

weights, bias = train(X, y, input_size, output_size, epochs, learning_rate)

Epoch 0, Loss: -0.5000
Epoch 100, Loss: -0.0599
Epoch 200, Loss: -0.0539
Epoch 300, Loss: -0.0514
Epoch 400, Loss: -0.0499
Epoch 500, Loss: -0.0492
Epoch 600, Loss: -0.0488
Epoch 700, Loss: -0.0484
Epoch 800, Loss: -0.0481
Epoch 900, Loss: -0.0480
Epoch 1000, Loss: -0.0484
Epoch 1100, Loss: -0.0490
Epoch 1200, Loss: -0.0495
Epoch 1300, Loss: -0.0496
Epoch 1400, Loss: -0.0497
Epoch 1500, Loss: -0.0496
Epoch 1600, Loss: -0.0496
Epoch 1700, Loss: -0.0496
Epoch 1800, Loss: -0.0495
Epoch 1900, Loss: -0.0495
Epoch 2000, Loss: -0.0495
Epoch 2100, Loss: -0.0494
Epoch 2200, Loss: -0.0494
Epoch 2300, Loss: -0.0494
Epoch 2400, Loss: -0.0494
Epoch 2500, Loss: -0.0494
Epoch 2600, Loss: -0.0493
Epoch 2700, Loss: -0.0493
Epoch 2800, Loss: -0.0493
Epoch 2900, Loss: -0.0493


In [430]:
predictions = predict(X, weights, bias)
print("Predicciones:")
print(predictions)

# Salidas reales para comparación
print("Salidas reales:")
print(y)

Predicciones:
[[3.01265421e-05 1.31076596e-05 8.03053946e-09]
 [1.01774799e-05 1.62561091e-10 1.07990401e-08]
 [4.07766562e-08 5.70824680e-07 5.06925068e-04]
 ...
 [8.90567252e-12 4.36195796e-08 5.67173605e-10]
 [4.79657074e-12 9.86433307e-01 1.99342553e-04]
 [2.23071010e-13 5.07973521e-19 5.92809146e-09]]
Salidas reales:
[[-0.66390084  1.34145628 -0.71274119]
 [ 1.50624903 -0.74545851 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]
 ...
 [-0.66390084  1.34145628 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]]


In [431]:
mse = np.mean((y - predictions) ** 2)
mse

0.9684770988075814

Bonus 1

Para agregar  k capas ocultas con n neuronas cada una hay que hacer modificaciones en la funcion en la que se crean los weights y los bias, al igual que en la de forward propagation. Además, yo voy a usar otra funcion de activacion(ReLu) para las capas intermedias. 

In [432]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [433]:
def bonus_initialize_weights(layer_sizes):
    weights = []
    biases = []
    for i in range(len(layer_sizes) - 1):
        w = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.01
        b = np.zeros((1, layer_sizes[i + 1]))
        weights.append(w)
        biases.append(b)
    return weights, biases  

In [434]:
def bonus_forward_pass(X, weights, biases, layer_sizes):
    activations = [X]  # Activaciones de todas las capas excepto al salida
    for i in range(len(layer_sizes) - 2):  # Capas ocultas
        z = np.dot(activations[-1], weights[i]) + biases[i]
        a = relu(z)
        activations.append(a)
    
    # Capa de salida con Sigmoid
    z = np.dot(activations[-1], weights[-1]) + biases[-1]
    output = sigmoid(z)
    activations.append(output)
    
    return activations

In [435]:
def bonus_backpropagation(X, y, activations, weights, biases, learning_rate):
    m = y.shape[0]  # Número de ejemplos
    L = len(weights)  # Número de capas
    
    delta_z = activations[-1] - y  # Error de la capa de salida
    delta_weights = np.dot(activations[-2].T, delta_z) / m  # Ajustado para el cálculo directo
    delta_bias = np.sum(delta_z, axis=0, keepdims=True) / m 

    # delta_weights = np.dot(activations[-2].T, delta_z) / m 
    # delta_bias = np.sum(delta_z, axis=0, keepdims=True) / m 


    # Actualizar pesos y sesgos de la capa de salida
    weights[-1] -= learning_rate * delta_weights
    biases[-1] -= learning_rate * delta_bias

    for l in range(L - 2, -1, -1):  # Desde la última capa oculta (L-2) hasta la de entrada (0)

        delta_z = np.dot(delta_z, weights[l + 1].T) * relu_derivative(activations[l +1 ])
    
        delta_weights = np.dot(activations[l].T, delta_z) / m
        delta_bias = np.sum(delta_z, axis=0, keepdims=True) / m

        # delta_weights = np.dot(activations[l].T, delta_z)
        # delta_bias = np.sum(delta_z, axis=0, keepdims=True)
    
        # Actualizar pesos y sesgos
        weights[l] -= learning_rate * delta_weights
        biases[l] -= learning_rate * delta_bias

    return weights, biases

In [436]:
def bonus_train(X, y, layer_sizes, epochs=1000, learning_rate=0.01):
    # Inicializar pesos y sesgos
    weights, biases = bonus_initialize_weights(layer_sizes)
    
    
    for epoch in range(epochs):
        # Propagación hacia adelante
        activations = bonus_forward_pass(X, weights, biases, layer_sizes)
        
        # Calcular pérdida
        loss = linear_error(y, activations[-1])
        
        # Retropropagación
        weights, biases = bonus_backpropagation(X, y, activations, weights, biases, learning_rate)
        
        # Imprimir el progreso cada 100 épocas
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    return weights, biases

In [437]:
def bonus_predict(X, weights, biases, layer_sizes):
    activations = bonus_forward_pass(X, weights, biases, layer_sizes)
    return activations[-1]

In [438]:
input_size = X.shape[1]  # Número de características de entrada
output_size = y.shape[1]  # Número de clases de salida
layer_sizes = [input_size, 6,3,2, output_size]

epochs = 10000
learning_rate = 0.01
weights, biases = bonus_train(X, y, layer_sizes, epochs, learning_rate)

Epoch 0, Loss: -0.5000
Epoch 100, Loss: -0.3909
Epoch 200, Loss: -0.3116
Epoch 300, Loss: -0.2545
Epoch 400, Loss: -0.2129
Epoch 500, Loss: -0.1817
Epoch 600, Loss: -0.1578
Epoch 700, Loss: -0.1389
Epoch 800, Loss: -0.1238
Epoch 900, Loss: -0.1114
Epoch 1000, Loss: -0.1011
Epoch 1100, Loss: -0.0924
Epoch 1200, Loss: -0.0849
Epoch 1300, Loss: -0.0783
Epoch 1400, Loss: -0.0726
Epoch 1500, Loss: -0.0675
Epoch 1600, Loss: -0.0630
Epoch 1700, Loss: -0.0589
Epoch 1800, Loss: -0.0552
Epoch 1900, Loss: -0.0518
Epoch 2000, Loss: -0.0486
Epoch 2100, Loss: -0.0457
Epoch 2200, Loss: -0.0430
Epoch 2300, Loss: -0.0405
Epoch 2400, Loss: -0.0382
Epoch 2500, Loss: -0.0360
Epoch 2600, Loss: -0.0339
Epoch 2700, Loss: -0.0320
Epoch 2800, Loss: -0.0302
Epoch 2900, Loss: -0.0285
Epoch 3000, Loss: -0.0269
Epoch 3100, Loss: -0.0254
Epoch 3200, Loss: -0.0239
Epoch 3300, Loss: -0.0226
Epoch 3400, Loss: -0.0214
Epoch 3500, Loss: -0.0202
Epoch 3600, Loss: -0.0191
Epoch 3700, Loss: -0.0181
Epoch 3800, Loss: -0.017

In [439]:
bonus_predictions = bonus_predict(X, weights, biases, layer_sizes)
print("Predicciones:")
print(bonus_predictions)

print("Salidas reales:")
print(y)

Predicciones:
[[0.00216756 0.00213527 0.00215983]
 [0.00216755 0.00213527 0.00215983]
 [0.00216755 0.00213527 0.00215983]
 ...
 [0.00216755 0.00213526 0.00215982]
 [0.00216754 0.00213526 0.00215982]
 [0.00216757 0.00213528 0.00215984]]
Salidas reales:
[[-0.66390084  1.34145628 -0.71274119]
 [ 1.50624903 -0.74545851 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]
 ...
 [-0.66390084  1.34145628 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]]


Bonus 2


Lo que yo haría para que la salida sea mutualmente excluyente es cambiar la funcion de activación final. En vez de usar sigmoide, yo utilizaría SoftMax. Lo que esta funcion de activacion hace es que la suma de todas las columnas de la salida den 1. Al hacer esto, se puede interpretar como la probabilidad de que pertenezca a cada clase.

 En el dataset de tipos de conductor por ejemplo, haria que en vez de devolverme un rango entre 0 y 1 para cada posiblidad (Commuter, Long-Distance Traveler) me devuelva numeros que sumando todas las columnas den 1, haciendo que cada uno represente la posibilidad de que el conjunto original (X) perteneza a cada columna. Una vez tenga las "probabilidades", la que mayor chance tenga será a la que pertenece.